In [1]:
# 초기코드 - python
from keras.utils import np_utils
from keras.layers import Dense
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,ZeroPadding2D,GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
#from PIL import Image
import os, glob, numpy as np
from keras.models import load_model
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Activation, Dense
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB2

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
!unzip /content/gdrive/MyDrive/LPD_competition.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/968/32.jpg        
   creating: train/486/
  inflating: train/486/21.jpg        
  inflating: train/486/22.jpg        
  inflating: train/486/23.jpg        
  inflating: train/486/12.jpg        
  inflating: train/486/13.jpg        
  inflating: train/486/14.jpg        
  inflating: train/486/15.jpg        
  inflating: train/486/16.jpg        
  inflating: train/486/17.jpg        
  inflating: train/486/18.jpg        
  inflating: train/486/19.jpg        
  inflating: train/486/20.jpg        
  inflating: train/486/9.jpg         
  inflating: train/486/10.jpg        
  inflating: train/486/11.jpg        
  inflating: train/486/0.jpg         
  inflating: train/486/1.jpg         
  inflating: train/486/2.jpg         
  inflating: train/486/3.jpg         
  inflating: train/486/4.jpg         
  inflating: train/486/5.jpg         
  inflating: train/486/6.jpg         
  inflating: train/486/7.jpg         
  inflating: train/486/8.jpg

In [4]:
train_data_dir = '/content/train'
img_height=260
img_width=260
batch_size=32

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    rotation_range=20, 
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.4,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.9,1.1),
    validation_split=0.2) # set validation split

val_datagen = ImageDataGenerator(
    #rescale=1./255,
     validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=True,
    subset='training') # set as training data

validation_generator = val_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=False,
    subset='validation') # set as validation data

Found 39000 images belonging to 1000 classes.
Found 9000 images belonging to 1000 classes.


In [5]:
effi = EfficientNetB2(include_top=False, input_shape=(260,260,3))
model=Sequential()
model.add(effi)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(1000,activation='softmax'))
model.summary()

# model = EfficientNetB4(include_top=True, input_shape=(380,380,3))
# model.summary()

31793152/31790344 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb2 (Functional)  (None, 9, 9, 1408)        7768569   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1408)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1408)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              1409000   
Total params: 9,177,569
Trainable params: 9,109,994
Non-trainable params: 67,575
_________________________________________________________________


# 새 섹션

In [6]:
#optimizer = SGD(lr=1e-4, momentum=0.9)
#optimizer RMSprop(lr=2e-5)
optimizer = keras.optimizers.Adam(lr=0.00004, decay=1e-6)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_dir = './model'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_path = model_dir + '/efficientnetB2_dropout12.model'
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [ ]:
nb_epochs=50
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint,early_stopping]
    )

hist_df = pd.DataFrame(history.history) 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
!cp history.csv /content/gdrive/MyDrive/history.csv
!cp -r model /content/gdrive/MyDrive/model

Epoch 1/50
1218/1218 [==============================] - 1123s 885ms/step - loss: 6.7308 - accuracy: 0.0172 - val_loss: 4.5603 - val_accuracy: 0.4702

Epoch 00001: val_loss improved from inf to 4.56031, saving model to ./model/efficientnetB2_dropout12.model
INFO:tensorflow:Assets written to: ./model/efficientnetB2_dropout12.model/assets
Epoch 2/50
1218/1218 [==============================] - 1066s 875ms/step - loss: 4.3293 - accuracy: 0.3833 - val_loss: 1.7280 - val_accuracy: 0.8732

Epoch 00002: val_loss improved from 4.56031 to 1.72801, saving model to ./model/efficientnetB2_dropout12.model
INFO:tensorflow:Assets written to: ./model/efficientnetB2_dropout12.model/assets
Epoch 3/50
1218/1218 [==============================] - 1070s 878ms/step - loss: 2.1378 - accuracy: 0.7477 - val_loss: 0.6614 - val_accuracy: 0.9440

Epoch 00003: val_loss improved from 1.72801 to 0.66136, saving model to ./model/efficientnetB2_dropout12.model
INFO:tensorflow:Assets written to: ./model/efficientnetB2_d

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

# 새 섹션

In [ ]:
#또는
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

!cp history.json ./gdrive/MyDrive/_history.json
!cp history.csv ./gdrive/MyDrive/_history.csv

In [ ]:
!mkdir temp
!mv ./test/* ./temp
!mkdir ./test/unknown
!mv ./temp/* ./test/unknown/

In [ ]:
test_data_dir = '/content/test'
test_datagen = ImageDataGenerator()
    #rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode=None, #결과 똑같이나온다 None이나 Categorical 이나
    color_mode='rgb',
    interpolation='lanczos',
    shuffle=False
    )

Found 72000 images belonging to 1 classes.


In [ ]:
co=0
st=train_generator.class_indices.keys()
result_arr=list(st)
print(result_arr)

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '22

In [ ]:
# !rm -rf /content/test/unknown
# !mkdir /content/test/unknown

In [ ]:
model = load_model('/content/model/efficientnetB2_dropout12.model')
prediction = model.predict(test_generator)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
predicted_class = np.argmax(prediction,axis=1)

In [ ]:
test = pd.read_csv('/content/sample.csv')
df = test.reindex(index=test.index[:72000], columns=list(test.columns))
print(df)

        filename  prediction
0          0.jpg         NaN
1          1.jpg         NaN
2          2.jpg         NaN
3          3.jpg         NaN
4          4.jpg         NaN
...          ...         ...
71995  71995.jpg         NaN
71996  71996.jpg         NaN
71997  71997.jpg         NaN
71998  71998.jpg         NaN
71999  71999.jpg         NaN

[72000 rows x 2 columns]


In [ ]:
co=0
for i in predicted_class :
    ans = i
    ans=result_arr[ans]
    ans=int(ans)
    fname =test_generator.filenames[co]
    fname = fname.replace("unknown/","")
    find_row=test.loc[(test['filename']==fname)]
    find_row=int(find_row.index.tolist()[0])
    df.loc[find_row,'prediction']=ans
    co+=1
    print(co)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
67001
67002
67003
67004
67005
67006
67007
67008
67009
67010
67011
67012
67013
67014
67015
67016
67017
67018
67019
67020
67021
67022
67023
67024
67025
67026
67027
67028
67029
67030
67031
67032
67033
67034
67035
67036
67037
67038
67039
67040
67041
67042
67043
67044
67045
67046
67047
67048
67049
67050
67051
67052
67053
67054
67055
67056
67057
67058
67059
67060
67061
67062
67063
67064
67065
67066
67067
67068
67069
67070
67071
67072
67073
67074
67075
67076
67077
67078
67079
67080
67081
67082
67083
67084
67085
67086
67087
67088
67089
67090
67091
67092
67093
67094
67095
67096
67097
67098
67099
67100
67101
67102
67103
67104
67105
67106
67107
67108
67109
67110
67111
67112
67113
67114
67115
67116
67117
67118
67119
67120
67121
67122
67123
67124
67125
67126
67127
67128
67129
67130
67131
67132
67133
67134
67135
67136
67137
67138
67139
67140
67141
67142
67143
67144
67145
67146
67147
67148
67149
67150
67151
67152
67153
67154
67155
67156
67157
67158
67159
67160
6716

In [ ]:
df.to_csv("result.csv",mode='w',index=False)
!cp result.csv ./gdrive/MyDrive/result.csv

# 새 섹션